### Invoke AWS Lambda from jupyter notebook

In [1]:
import pandas as pd
import json
import boto3

In [ ]:
# Fetch the aws user region
my_session = boto3.session.Session()
my_region = my_session.region_name

In [ ]:
# Initialize AWS Lambda client
lambda_client = boto3.client('lambda', region_name=my_region)

In [ ]:
# Define the AWS Lambda function name
function_name = 'my_aws_lambda'

In [ ]:
# Define the function to invoke the AWS Lambda function on the desired column
def invoke_lambda(json_string):
    
    payload = json.loads(json_string)
    
    # Invoke the AWS Lambda function
    response = lambda_client.invoke(FunctionName=function_name, Payload=json.dumps(payload))
    
    # Extract the output from the response
    output_data = json.loads(response['Payload'].read().decode('utf-8'))
    
    # Store the output data in a new column
    return output_data['result']

In [ ]:
# Load the dataframe
df = pd.read_csv('inputs.csv')
print(df.shape)
df.head()

In [ ]:
# Apply the function to the desired column
df['prediction'] = df['input_features_json'].apply(invoke_lambda)

### Assumption: 
- You have already created an AWS Lambda function named *my_aws_lambda* 
- Above example uses following files while registering ECR for lambda

`lambda_function.py`
***
```python
import pickle
import json
import pandas as pd

# Load the saved preprocessor 
BASEPATH = '.'
PREPROCESS_FILENAME = BASEPATH + '/Preprocessor.sav'
preprocess = pickle.load(open(PREPROCESS_FILENAME, 'rb'))

# Load the trained ML model 
MODEL_FILENAME = BASEPATH + '/trainedML_model.sav'
model = pickle.load(open(MODEL_FILENAME, 'rb'))


def get_prediction(json_string):
    try:
        inputs_features_dict = json.loads(json_string)
        df_features = pd.DataFrame([inputs_features_dict])
        
        scaled_features = preprocess.transform(df_features)
        prediction = model.predict(scaled_features) 
    
    except Exception as e:
        prediction = -1
        print(f'Exception: {e}')
    return prediction


def lambda_handler(event, context):
    result = get_income(json.dumps(event))
    return {
        "statusCode": 200, 
        "result": result, 
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'        
        }
    }
```
***

`Dockerfile`

```yaml
FROM public.ecr.aws/lambda/python:3.6
RUN pip install numpy scikit-learn pandas
# ENTRYPOINT ["/lambda-entrypoint.sh"]
COPY Preprocessor.sav ./
COPY trainedML_model.sav ./
COPY lambda_function.py ./
CMD ["lambda_function.lambda_handler"]
```
***